In [204]:
%autoawait

IPython autoawait is `on`, and set to use `asyncio`


In [205]:
import asyncio
import aiohttp

In [206]:
sess = aiohttp.ClientSession("https://restapi.map.360.cn", headers={
    "user-agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/123.0.0.0 Safari/537.36 Edg/123.0.0.0"
})

Unclosed client session
client_session: <aiohttp.client.ClientSession object at 0x00000294180B9220>


In [207]:
from urllib.parse import quote
import time

async def poi_search_360(sess: aiohttp.ClientSession, keyword: str, city: str) -> dict:
    params = {
        "keyword": keyword,
        "cityname": city,
        "d": "pc",
        "brand_cpc": "on",
        "batch": 1,
        "number": 10,
        "qii": "true",
        "scheme": "https",
        "regionType": "rectangle",
        "sid": 1000,
        "ua": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/123.0.0.0 Safari/537.36 Edg/123.0.0.0",
        "browser_size": "1232,1059",
        "screen_size": "2560,1440",
        "screen_pixel_ratio": 1,
        "_": time.time_ns()//10**6
    }
    uri = "/newapi"
    # uri = "/app/pit"
    params_str = '&'.join(
        [f"{key}={value}" for key, value in sorted(params.items())])
    uri = f"{uri}?{params_str}"
    async with sess.get(uri) as resp:
        if resp.status == 200:
            return await resp.json()

In [221]:

(await poi_search_360(sess, "酒店", "东莞"))["poi"][0]

{'cat_new_name': '酒店',
 'name': '7天连锁酒店(东莞虎门步行街黄河时装城店)',
 'avg_price': 91,
 'avg_price_m': 91,
 'type': '住宿;酒店;宾馆酒店;经济型酒店;快捷酒店;星级酒店;二星级酒店',
 'cat': 0,
 'category': 'hotel',
 'tags': '住宿|7天连锁酒店|宾馆酒店|快捷酒店|酒店|经济型酒店|星级酒店|宾馆|连锁酒店|二星级酒店',
 'cat_new_path': '4,30',
 'avg_rating': 4.9,
 'x': 113.673473,
 'b_area_name': '虎门步行街',
 'primaryid': '8ef4c9131c181ea5',
 'total_cnt': 0,
 'is_near_bus': 0,
 'recall': 0,
 'children': None,
 'address': '东莞市长德路2号太平广场电子城',
 'postcode': '523000',
 'tels': ['0769-82889977'],
 'name_wipe': '7天连锁酒店',
 'tel': '0769-82889977',
 'datasource': '0',
 'datasourceid': '',
 'pguid': '8ef4c9131c181ea5',
 'cityid': '441900',
 'area': '东莞市',
 'citycode': '',
 'detail': {'zflag': 4,
  'primary': 'elong',
  'clz': '住宿服务;宾馆酒店',
  'poi_clz': '住宿;经济型酒店',
  'poi_type': '3',
  'poi_main_cat_new': 30,
  'update_master_time': 1651893023,
  'update_bada_time': 1712589065,
  's_photo_url': 'https://so5.360tres.com/t0130dee2ca996a1083.jpg',
  'book_url': 'https://m.elong.com/Hotel/Det

In [222]:
baidu_sess = aiohttp.ClientSession("https://map.baidu.com", headers={
    "user-agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/123.0.0.0 Safari/537.36 Edg/123.0.0.0"
})

In [240]:

import json
import functools

async def poi_search_baidu(sess: aiohttp.ClientSession, keyword: str, city: str) -> dict:
    resp = await sess.get(f"/?qt=cur&wd={city}")
    data = json.loads(await resp.read())
    if "content" not in data:
        return None
    cid = data["content"]["area"]
    resp = await sess.get(f"/?qt=spot&c={cid}&wd={keyword}")
    return json.loads(await resp.read())



# tasks = []
# for i in range(1):
#     tasks.append(baidu_poi(baidu_sess))
# import asyncio
# await asyncio.gather(*tasks)
# from urllib.parse import unquote
# unquote("/?newmap=1&reqflag=pcmap&biz=1&from=webmap&da_par=direct&pcevaname=pc4.1&qt=spot&from=webmap&c=138&wd=%E7%BE%8E%E9%A3%9F&wd2=&pn=0&nn=0&db=0&sug=0&addr=0&pl_data_type=cater&pl_sub_type=%E9%A4%90%E9%A6%86&pl_price_section=0%2C%2B&pl_sort_type=data_type&pl_sort_rule=0&pl_discount2_section=0%2C%2B&pl_groupon_section=0%2C%2B&pl_cater_book_pc_section=0%2C%2B&pl_hotel_book_pc_section=0%2C%2B&pl_ticket_book_flag_section=0%2C%2B&pl_movie_book_section=0%2C%2B&pl_business_type=cater&pl_business_id=&da_src=pcmappg.poi.page&on_gel=1&src=7&gr=3&l=13&rn=50&tn=B_NORMAL_MAP&auth=uxNxzRBTVTTtVLF8LXV8FzRyC1WbB9AvkGcuVtvvhguVtvyheuVtvCMGuVtcvY1SGpuNtCH7RB9AvIPcuxtw8wkv7uvZgMuVtv%40vcuVtvc3CuVtvcPPuVtveGvuxEtHrvU2eGvh3CuVtvhgMuzVVtvrMhuxt2dd9dvyuRyE%3D6150I%3DhU1eE%40JJTY8I3P25520DZ5vyOyASCU77SulnDjnCETEGXHZ%40%40X&seckey=pENQXAnHOCZbhsbcNwch%2BkvBzq6t4gu%2FYOMuKkX%2Fa5E%3D%2CXfJucry29BBy2LreQq7HNcnNwdIuyHl5DM7RMMtinhiFlgvqPjQJdXMR8g1zAj2x6VpskU_3f0pTYeVoMAYU9s2Q_ti8pso3r0dtExuqFkzl6rqIEbgNNQNxp6a6UWRITzS_4wEZlHaIUNUoWy2G-cSPfcMGeSxm6cHJxQvHvvb_j1vtu1tSZIXGxEDQ954u&u_loc=12610094,2594516&ie=utf-8&b=(12572902,2576844;12653286,2618188)&t=1712839100052&newfrom=zhuzhan_webmap")

res = await poi_search_baidu(baidu_sess, "停车场", "深圳")
# res["content"][0]["x"]/10**7, res["content"][0]["y"]/10**7
res["content"][0]

{'acc_flag': 0,
 'addr': '深圳市盐田区',
 'addr_hot_percent': 0,
 'address_color_shows': [],
 'address_norm': '[广东省(440000)|PROV|0|][深圳市(440300)|CITY|1|][盐田区(440308)|AREA|1|]',
 'admin_info': {'area_id': 440308,
  'area_name': '盐田区',
  'city_id': 440300,
  'city_name': '深圳市',
  'province_id': 440000,
  'province_name': '广东省',
  'town_id': 440308002,
  'town_name': '盐田街道'},
 'alias': ['大华兴寺停车场'],
 'api_admin_info': {'area_code': 1612,
  'area_name': '盐田区',
  'city_code': 340,
  'city_name': '深圳市',
  'prov_code': 7,
  'prov_name': '广东省'},
 'area': 1612,
 'area_name': '深圳市盐田区',
 'biz_type': 0,
 'brand_id': None,
 'catalogID': 217,
 'city_id': 340,
 'city_name': '深圳市',
 'cla': [[22, '交通设施'], [217, '<font color="#c60a00">停车场</font>/停车区']],
 'class_similar_score': 0.90454262495041,
 'click_flag': 0,
 'cqr': 1,
 'detail': 1,
 'diPointX': 1272242890,
 'diPointY': 257153218,
 'di_tag': '交通设施 停车场 地上停车场',
 'dis': -1,
 'dist2route': 0,
 'dist2start': 0,
 'ernie_score': 0,
 'ext_display': {'display_info'